# Recursive data types and functions

## Recursive types

### The `List` type

Lists are data structures which represent sequences of values of the same type, of finite length. They can be defined recursively in an informal way as follows: 
- A list is the empty sequence
- A list is a non-empty sequence made of a value and another list, which represent the head and tail of the list, respectively

Thus, the type `IntList`, which represents lists of integers, must satisfy the following algebraic equation:

`IntList = 1 + Int * IntList`

i.e., a list of integers is the empty sequence (represented by the singleton type `1`), or an integer (the head) and a list (its tail).



 Option[(Int, IntList)] = 
    Either[Unit,(Int, IntList)]

The implementation in Scala is similar to the following one (we also give the generic version `List[A]`, rather than the implementation of `IntList`):

In [1]:
object StdDefinition {
    sealed abstract class Option[+A]
    case object None extends Option[Nothing]
    case class Some[A](a: A) extends Option[A]
    
    sealed abstract class List[A]
    case class Empty[A]() extends List[A]
    case class NonEmpty[A](head: A, tail: List[A]) 
        extends List[A]
    
}

defined object StdDefinition

However the actual implementation of [immutable lists](https://github.com/scala/scala/blob/v2.13.1/src/library/scala/collection/immutable/List.scala#L79) in the standard library of Scala defines the empty list as an object, rather than a class. This forces us to declare the list covariantly in its generic parameter `A`, which is somewhat inconvenient at times.  The standard definition looks like as follows:

In [ ]:
object ActualStdDefinition{
    sealed abstract class List[A]
    case object Nil extends List[Nothing]
    case class ::[A](head: A, tail: List[A]) 
        extends List[A]
}

### Some syntactic sugar

Note that we can write standard lists with a more compact syntax: 

In [6]:
// Less beautifully 

::(1, ::(2, ::(3, Nil)))

// More idiomatically

1 :: 2 :: 3 :: Nil

List(1,2,3)


res5_0: ::[Int] = List(1, 2, 3)
res5_1: List[Int] = List(1, 2, 3)
res5_2: List[Int] = List(1, 2, 3)

And can also pattern match lists, similarly:

In [7]:
// Less beautifully

val l1: List[Int] = 1 :: 2 :: 3 :: Nil

l1 match {
    case Nil => "vacío"
    case ::(head, tail) => "no vacío"
}
// more idiomatically

l1 match {
    case Nil => 1
    case head :: tail => 2
}

// or

l1 match {
    case List() => 1
    case List(e1,e2,e3) => 3
    case _ => 4
}


l1: List[Int] = List(1, 2, 3)
res6_1: String = "no vac\u00edo"
res6_2: Int = 2
res6_3: Int = 3

##  Recursive functions

Since lists are defined recursively, functions over lists will be commonly recursive as well. For instance, let's implement a recursive function that computes the length of a list. But before, let's implement the function imperatively for the sake of comparison:

In [24]:
// Using mutable variables

def lengthI[A](list: List[A]): Int = {
    var out: Int = 0
    var aux: List[A] = list
    while (aux != Nil) {
        out = out + 1
        aux = aux.tail
    }
    out
}
    

defined function lengthI

In [10]:
length(List(1,2,3,4))
length(List())

res9_0: Int = 4
res9_1: Int = 0

The recursive function is implemented as follows: 

In [11]:
// Using recursive functions

def length[A](list: List[A]): Int = 
    list match {
        case Nil => 0 : Int
        case head :: tail => 
            val tailLength: Int = length(tail)
            1 + tailLength : Int 
    }

defined function length

In [23]:
// Using recursive functions

def lengthR[A](list: List[A]): Int = 
    list match {
        case Nil => 0
        case _ :: tail => 
            1 + lengthR(tail)
    }

defined function lengthR

In [14]:
List.fill(50)('a')

res13: List[Char] = List(
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
  'a',
...

In [18]:
length(List.fill(100000)(1))

: 

Some comments: 
- The recursive function is implemented in a _type-driven development_ style: we proceed, step-by-step, analysing the types of input data that we have available so far, and the types of output that we have to generate. This leads to a divide-and-conquer problem solving strategy and hugely facilitates the implementation.
- The recursive function is less efficient, since the stack will blow up with lists of enough lenght.

### Tail-recursive functions

The implementation using tail-recursion solves the problems with the stack. It commonly makes use of auxiliary functions:

In [35]:
// Using tail-recursive functions


def lengthT[A](list: List[A]): Int = {
    
    def lengthAux(out: Int, aux: List[A]): Int = 
        aux match {
            case Nil => 0
            case _ :: tail => 
                lengthAux(out + 1, tail)
        }
    
    lengthAux(0, list)
/*    
    var out: Int = 0
    var aux: List[A] = list
    while (aux != Nil) {
        out = out + 1
        aux = aux.tail
    }
    out*/
}

defined function lengthT

In [36]:
length[Int](List.fill(10000000)(1))

res35: Int = 10000000

We can check the stack-safety problems of non-tail recursive functions, by calculating the length of a very big list. We will use the following function, which creates a constant list of given length.

In [36]:
// First, imperatively



In [36]:
// Next, tail-recursively



Now, let's calculate the length of a list long enough, using each of the three implementations:

In [36]:
// Imperatively


In [36]:
// Tail-recursive


In [36]:
// Plain recursive


### Unit testing with `scalatest`

In [37]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$                               

import org.scalatest._

From now on, we will also make extensive use of unit testing for the different functions that we implement. And we will use the [`scalatest`](http://www.scalatest.org/) library for that purpose. In particular, for each function we will implement a test catalogue that test it against different test cases. For instance, this is a possible test class for the `length` function:

In [38]:
class TestLength(len: List[Int] => Int) extends FlatSpec with Matchers{
    
    "length" should "work" in {
        len(List()) shouldBe 0
        len(List(1)) shouldBe 1
        len(List(1,2,3,4)) shouldBe 4
    }
}

defined class TestLength

The method `shouldBe` is a _matcher_. The scalatest library offers an extensive catalogue of [them](http://www.scalatest.org/user_guide/using_matchers). Similarly, scalatest also support many different [testing styles](http://www.scalatest.org/user_guide/selecting_a_style). The chosen one here was `FlatSpec`. In order to execute the test catalogue we can simply use the scalatest method `run`:

In [40]:
run(new TestLength(lengthI))
run(new TestLength(lengthR))
run(new TestLength(lengthT[Int] _))

cmd37$Helper$TestLength:
length
- should work
cmd37$Helper$TestLength:
length
- should work
cmd37$Helper$TestLength:
length
- should work *** FAILED ***
  0 was not equal to 1 (cmd37.sc:5)


### Example: adding numbers

Let's implement a function that sums all the numbers of a list.

In [42]:
class TestSum(
    sum: List[Int] => Int
) extends FlatSpec with Matchers{
    "sum" should "work" in {
        sum(List()) shouldBe 0
        sum(List(-1,-2,2,1)) shouldBe 0
        sum(List(1,2,3)) shouldBe 6
        sum(List(0,0,0,0,0)) shouldBe 0
    }
}

defined class TestSum

In [45]:
// Recursively

def sum(list: List[Int]): Int = 
    list match {
        case Nil => 
            0 : Int
        case head :: tail => 
            val tailSum: Int = sum(tail)
            head + tailSum : Int 
    }

defined function sum

In [47]:
// Recursively

def sum(list: List[Int]): Int = 
    list match {
        case Nil => 
            0
        case head :: tail => 
            head + sum(tail)
    }

defined function sum

In [46]:
run(new TestSum(sum))

cmd41$Helper$TestSum:
sum
- should work *** FAILED ***
  1 was not equal to 0 (cmd41.sc:5)


In [48]:
// With tail-recursion

def sum(list: List[Int]): Int = {
    
    def sumAux(out: Int, aux: List[Int]): Int = 
        aux match {
            case Nil => out
            case head :: tail => 
                sumAux(out + head, tail)
        }
    
    sumAux(0, list)
}

defined function sum

In [49]:
run(new TestSum(sum))

cmd41$Helper$TestSum:
sum
- should work


### Example: multiplying list elements

Let's multiply the elements of a list. If the list is empty we return the identity element for integers.

In [ ]:
object TestProduct extends FlatSpec with Matchers{
    "product" should "work" in {
    }
}

This is the common recursive implementation:

It works as expected: 

In [ ]:
run(TestProduct)

But we can optimize the function a little bit. Note that if the number 0 belongs to the list, then the result is 0, no matter how many elements the list has. So, once we find the element 0 it's a waste of resources to make the recursive call. Let's take this into account.

In [ ]:
/*def product(list: List[Int]): Int =
    list match {
        case Nil => 1
        case head :: tail => head * product(tail)
    }
    */

In [ ]:
run(TestProduct)

A similar optimization can be made for the tail-recursive implementation.

### Example: membership

Let's implement a function that given a list and an element, returns whether the element belongs to that list.

In [ ]:
object TestMember extends FlatSpec with Matchers{
    "member" should "work" in {
    }
}

In [ ]:
run(TestMember)

We can also pattern match against a specific value as follows:

### Example: last element

Let's implement a function that returns the last element of a given list. Note that an empty list does not have elements, and, hence, does not have a last element.

In [50]:
class TestLast(
    last: List[Int] => Option[Int]
) extends FlatSpec with Matchers{
    "last" should "work" in {
        last(List()) shouldBe None
        last(List(1,2,3)) shouldBe Some(3)
        last(List(1)) shouldBe Some(1)
    }
}

defined class TestLast

In [52]:
def last[A](list: List[A]): Option[A] = 
    list match {
        case Nil => 
            None : Option[A]
        case head :: tail => 
            val tailLast: Option[A] = last(tail)
            tail match {
                case Nil =>
                    Some(head) : Option[A]
                case head2 :: tail2 => 
                    tailLast : Option[A]
            }
    }

defined function last

In [57]:
def last[A](list: List[A]): Option[A] = 
    list match {
        case Nil => 
            None : Option[A]
        case head :: tail => 
            val tailLast: Option[A] = last(tail)
            tailLast match {
                case None => 
                    Some(head) : Option[A]
                case _ => 
                    tailLast : Option[A]
            }
    }

defined function last

In [52]:
def last[A](list: List[A]): Option[A] = 
    list match {
        case Nil => 
            None : Option[A]
        case head :: tail => 
            if (tail == Nil) Some(head) : Option[A]
            else last(tail) : Option[A]
    }

defined function last

In [55]:
def last[A](list: List[A]): Option[A] = 
    list match {
        case Nil => 
            None
        case head :: Nil => 
            Some(head)
        case _ :: tail => 
            last(tail)
    }

defined function last

In [58]:
run(new TestLast(last))

cmd49$Helper$TestLast:
last
- should work


### Example: insert last

Now, a function that allows us to insert an element at the end of the list. 

In [60]:
class TestInsertLast(
    insert: List[Int] => Int => List[Int]
) extends FlatSpec with Matchers{
    "insertLast" should "work" in {
        insert(List())(1) shouldBe List(1)
        insert(List(1,2,3,4))(5) shouldBe List(1,2,3,4,5)
    }
}

defined class TestInsertLast

In [63]:
def insertLast[A](list: List[A])(a: A): List[A] = 
    list match {
        case Nil => 
            List(a) : List[A]
        
        case head :: tail => 
            val tailInsertedLast: List[A] = 
                insertLast(tail)(a)
            
            head :: tailInsertedLast : List[A]
    }
    

defined function insertLast

In [64]:
run(new TestInsertLast(insertLast))

cmd59$Helper$TestInsertLast:
insertLast
- should work


### Example: reverse lists

Implement a function which receives a list and returns its reverse.

In [ ]:
object TestReverse extends FlatSpec with Matchers{
    "reverse" should "work" in {
    }
}

In [ ]:
// Recursively: Really inefficient 


In [ ]:
run(TestReverse)

In [ ]:
// Tail-recursive, efficiently


In [ ]:
run(TestReverse)

### Example: concatenate lists

Let's implement this function step-by-step, following the types. We start from the signature of the desired function:

In [ ]:
object TestConcatenate extends FlatSpec with Matchers{
    "concatenate" should "work" in {
    }
}

In [ ]:
// Recursive

In [ ]:
run(TestConcatenate)

In [ ]:
// Tail-recursive

In [ ]:
run(TestConcatenate)